## Making database for tool calling

In [1]:
import sys
import os
from dotenv import load_dotenv
import logging
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing import Dict, Any

# Get the absolute path of the project's root directory (one level up from 'Junk')
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added '{project_root}' to the Python path.")

# Now your regular imports will work correctly
from cx_dashboard.logger import logger
import logging
log = logging.getLogger(__name__)

log.info("Successfully imported modules from the parent directory.")

Added '/Users/aryanbhargava/CX_Management_project' to the Python path.
[ 2025-08-22 17:10:06,395 ] __main__ - INFO - Successfully imported modules from the parent directory.


In [2]:
import os
import logging as log
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Basic logging setup
log.basicConfig(level=log.INFO)

def setup_agent_tables():
    """
    Connects to the database, drops existing agent-related tables for a clean
    setup, creates new tables (users, purchase_history), and inserts sample data.
    """
    log.info("--- Starting Database Setup for Agent ---")
    
    # --- 1. Connect to the Database ---
    load_dotenv()
    db_user = os.getenv("DB_USER")
    db_password_raw = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    db_name = os.getenv("DB_NAME") # We'll use the main DB for this

    if not all([db_user, db_password_raw, db_host, db_port, db_name]):
        log.error("Database configuration is missing in .env file. Aborting.")
        return

    try:
        encoded_password = quote_plus(db_password_raw)
        connection_url = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"
        engine = create_engine(connection_url)
        
        with engine.connect() as connection:
            log.info("Successfully connected to the database.")
            
            # --- 2. Define SQL Commands ---
            # Use DROP TABLE IF EXISTS for a clean, re-runnable script
            # CORRECTED LINE: Added CASCADE to the DROP TABLE users command
            sql_commands = """
            DROP TABLE IF EXISTS purchase_history;
            DROP TABLE IF EXISTS users CASCADE;

            CREATE TABLE users (
                id SERIAL PRIMARY KEY,
                name VARCHAR(100),
                email VARCHAR(100) UNIQUE NOT NULL
            );

            CREATE TABLE purchase_history (
                id SERIAL PRIMARY KEY,
                user_id INTEGER NOT NULL,
                order_id VARCHAR(50) UNIQUE NOT NULL,
                asin VARCHAR(20) NOT NULL,
                purchase_date DATE NOT NULL,
                CONSTRAINT fk_user
                    FOREIGN KEY(user_id) 
                    REFERENCES users(id)
            );

            -- Insert Sample Data
            INSERT INTO users (id, name, email) VALUES 
            (1, 'Priya Sharma', 'priya@example.com'),
            (2, 'Amit Kumar', 'amit@example.com');

            -- Change the sequence to start after the manually inserted IDs
            ALTER SEQUENCE users_id_seq RESTART WITH 3;

            INSERT INTO purchase_history (user_id, order_id, asin, purchase_date)
            VALUES 
            (1, '123456789', 'B000ASDGK8', '2025-07-20'),
            (2, '112121007', 'B00176GSEI', '2025-06-15');
            """
            
            # --- 3. Execute the SQL ---
            log.info("Dropping old tables and creating new ones...")
            connection.execute(text(sql_commands))
            connection.commit()
            log.info("✅ Successfully created 'users' and 'purchase_history' tables.")
            log.info("✅ Successfully inserted sample data.")

    except Exception as e:
        log.error(f"An error occurred during database setup: {e}", exc_info=True)


def setup_task_tables():
    """
    Creates the tables required for task assignment.
    """
    log.info("--- Setting up Task Assignment tables ---")
    
    # --- 1. Connect to the Database ---
    load_dotenv()
    db_user = os.getenv("DB_USER")
    db_password_raw = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    db_name = os.getenv("DB_NAME") # We'll use the main DB for this

    if not all([db_user, db_password_raw, db_host, db_port, db_name]):
        log.error("Database configuration is missing in .env file. Aborting.")
        return
    
    try:
        encoded_password = quote_plus(db_password_raw)
        connection_url = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"
        engine = create_engine(connection_url)
        with engine.connect() as connection:
            # NOTE: For this script to work, the 'users' table must exist first.
            # It's good practice to ensure setup_agent_tables() runs before this.
            sql = """
            DROP TABLE IF EXISTS support_tasks;
            DROP TABLE IF EXISTS team_members;
            DROP TABLE IF EXISTS departments;

            CREATE TABLE departments (
                id SERIAL PRIMARY KEY,
                name VARCHAR(100) UNIQUE NOT NULL
            );

            CREATE TABLE team_members (
                id SERIAL PRIMARY KEY,
                name VARCHAR(100),
                email VARCHAR(100) UNIQUE,
                department_id INTEGER NOT NULL REFERENCES departments(id)
            );

            CREATE TABLE support_tasks (
                id SERIAL PRIMARY KEY,
                user_id INTEGER NOT NULL REFERENCES users(id),
                order_id VARCHAR(50),
                assigned_to_member_id INTEGER REFERENCES team_members(id),
                summary TEXT NOT NULL,
                department VARCHAR(50),
                status VARCHAR(20) DEFAULT 'open',
                created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
            );

            INSERT INTO departments (id, name) VALUES
            (1, 'Returns & Refunds'),
            (2, 'Technical Support');
            ALTER SEQUENCE departments_id_seq RESTART WITH 3;

            INSERT INTO team_members (id, name, email, department_id) VALUES
            (1, 'Rohan', 'rohan@support.com', 1),
            (2, 'Sunita', 'sunita@support.com', 2);
            ALTER SEQUENCE team_members_id_seq RESTART WITH 3;
            """
            connection.execute(text(sql))
            connection.commit()
            log.info("✅ Successfully created 'departments', 'team_members', and 'support_tasks' tables.")
    except Exception as e:
        log.error(f"An error occurred during task table setup: {e}", exc_info=True)

In [3]:
setup_task_tables()
setup_agent_tables()

[ 2025-08-22 17:10:24,845 ] root - INFO - --- Setting up Task Assignment tables ---
[ 2025-08-22 17:10:25,018 ] root - INFO - ✅ Successfully created 'departments', 'team_members', and 'support_tasks' tables.
[ 2025-08-22 17:10:25,019 ] root - INFO - --- Starting Database Setup for Agent ---
[ 2025-08-22 17:10:25,034 ] root - INFO - Successfully connected to the database.
[ 2025-08-22 17:10:25,035 ] root - INFO - Dropping old tables and creating new ones...
[ 2025-08-22 17:10:25,068 ] root - INFO - ✅ Successfully created 'users' and 'purchase_history' tables.
[ 2025-08-22 17:10:25,069 ] root - INFO - ✅ Successfully inserted sample data.


## Tools 

In [4]:

def get_db_connection():
    """Create and return a database connection."""
    db_user = os.getenv("DB_USER")
    db_password_raw = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    db_name = os.getenv("DB_NAME")

    if not all([db_user, db_password_raw, db_host, db_port, db_name]):
        log.error("Database configuration is missing in .env file.")
        return None

    try:
        encoded_password = quote_plus(db_password_raw)
        connection_url = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"
        engine = create_engine(connection_url)
        return engine
    except Exception as e:
        log.error(f"Failed to create database connection: {e}")
        return None

def verify_order_id_into_db(order_id: str) -> Dict[str, Any]:
    """
    Verify if an order ID exists in the purchase_history table.
    Returns a dictionary with verification status and order details.
    """
    log.info(f"Verifying order ID: {order_id}")
    
    engine = get_db_connection()
    if not engine:
        return {"valid": False, "error": "Database connection failed"}
    
    try:
        with engine.connect() as connection:
            # Query to check if order exists and get user details
            query = text("""
                SELECT ph.order_id, ph.asin, ph.purchase_date, u.name, u.email, u.id as user_id
                FROM purchase_history ph
                JOIN users u ON ph.user_id = u.id
                WHERE ph.order_id = :order_id
            """)
            
            result = connection.execute(query, {"order_id": order_id})
            order_data = result.mappings().first()
            
            if order_data:
                log.info(f"Order ID {order_id} verified successfully")
                return {
                    "valid": True,
                    "order_id": order_data["order_id"],
                    "asin": order_data["asin"],
                    "purchase_date": order_data["purchase_date"].isoformat() if order_data["purchase_date"] else None,
                    "customer_name": order_data["name"],
                    "customer_email": order_data["email"],
                    "user_id": order_data["user_id"]
                }
            else:
                log.warning(f"Order ID {order_id} not found in database")
                return {"valid": False, "error": "Order ID not found"}
                
    except Exception as e:
        log.error(f"Error verifying order ID {order_id}: {e}")
        return {"valid": False, "error": f"Database error: {str(e)}"}
    
    finally:
        engine.dispose()

def log_complaint_to_db_full(order_id: str, department: str, complaint_details: str, complaint_summary: str) -> Dict[str, Any]:
    """
    Log a complaint to the support_tasks table in the database.
    Returns a dictionary with logging status and task details.
    """
    log.info(f"Logging complaint for order {order_id} to {department} department")
    
    # First verify the order exists
    order_verification = verify_order_id_into_db(order_id)
    if not order_verification["valid"]:
        return {
            "success": False,
            "error": f"Cannot log complaint: {order_verification.get('error', 'Order verification failed')}",
            "task_id": None
        }
    
    engine = get_db_connection()
    if not engine:
        return {"success": False, "error": "Database connection failed", "task_id": None}
    
    try:
        with engine.connect() as connection:
            # Get user_id from order verification
            user_id = order_verification["user_id"]
            
            # Find an appropriate team member for the department
            team_member_query = text("""
                SELECT tm.id, tm.name, tm.email
                FROM team_members tm
                JOIN departments d ON tm.department_id = d.id
                WHERE d.name ILIKE :department
                LIMIT 1
            """)
            
            team_member_result = connection.execute(
                team_member_query, 
                {"department": f"%{department}%"}
            )
            team_member = team_member_result.mappings().first()
            
            assigned_to = team_member["id"] if team_member else None
            
            # Insert the complaint into support_tasks
            insert_query = text("""
                INSERT INTO support_tasks 
                (user_id, order_id, assigned_to_member_id, summary, department, status)
                VALUES (:user_id, :order_id, :assigned_to, :summary, :department, 'open')
                RETURNING id, created_at
            """)
            
            result = connection.execute(
                insert_query,
                {
                    "user_id": user_id,
                    "order_id": order_id,
                    "assigned_to": assigned_to,
                    "summary": complaint_summary,
                    "department": department
                }
            )
            
            task_data = result.mappings().first()
            connection.commit()
            
            log.info(f"Complaint logged successfully for order {order_id}, task ID: {task_data['id']}")
            
            return {
                "success": True,
                "task_id": task_data["id"],
                "created_at": task_data["created_at"].isoformat(),
                "assigned_to": team_member["name"] if team_member else "Unassigned",
                "department": department,
                "order_details": order_verification
            }
            
    except Exception as e:
        log.error(f"Error logging complaint for order {order_id}: {e}")
        connection.rollback()
        return {"success": False, "error": f"Database error: {str(e)}", "task_id": None}
    
    finally:
        engine.dispose()

In [5]:
from typing import Literal, Optional
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, StateGraph
from langgraph.types import Command
from typing_extensions import Annotated, TypedDict
from pydantic import BaseModel, Field

# Graph state
class State(TypedDict):
    """Customer service state."""
    messages: list
    order_id: str | None
    order_id_verified: bool | None
    complaint_details: str | None
    complaint_summary: str | None
    department: str | None
    complaint_logged: bool | None

# Classification schema
class DepartmentClassification(BaseModel):
    """Classification of the complaint department."""
    department: Literal["Billing", "Technical Support", "Product Quality", "General Inquiry"] = Field(
        description="The department that should handle this complaint"
    )

# Define tools (these would be implemented with your actual backend)
def verify_order_id(order_id: str) -> bool:
    """
    Verify if an order ID is valid.
    Compatible with your existing agent's expected return type.
    """
    result = verify_order_id_into_db(order_id)
    return result["valid"]

def log_complaint_to_db(order_id: str, department: str, complaint_details: str) -> bool:
    """
    Log a complaint to the database.
    Compatible with your existing agent's expected return type.
    """
    # Generate a summary (using your existing summarization approach)
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import HumanMessage
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    summary_prompt = f"Summarize this complaint in 1-2 sentences: {complaint_details}"
    response = llm.invoke([HumanMessage(content=summary_prompt)])
    complaint_summary = response.content
    
    # Log to database
    result = log_complaint_to_db_full(order_id, department, complaint_details, complaint_summary)
    return result["success"]

# Initialize the chat model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create a classifier for department classification
classifier_llm = llm.with_structured_output(DepartmentClassification)

def extract_order_id(text: str) -> str | None:
    """Extract order ID from text."""
    words = text.split()
    for word in words:
        # Remove common punctuation
        clean_word = word.strip(".,!?;:")
        if clean_word.isdigit() and len(clean_word) >= 6:
            return clean_word
    return None

def check_order_id(state: State) -> Command[Literal["verify_order", "__end__"]]:
    """Check if we have an order ID and route accordingly."""
    # Check if we already have a verified order ID
    if state.get("order_id_verified"):
        return Command(goto="ask_details")
    
    # Extract order ID from the last user message
    order_id = None
    for msg in reversed(state["messages"]):
        if isinstance(msg, dict) and msg.get("role") == "user":
            order_id = extract_order_id(msg["content"])
            break
        elif hasattr(msg, 'type') and msg.type == "human":
            order_id = extract_order_id(msg.content)
            break
    
    if order_id:
        # We have an order ID, need to verify it
        return Command(update={"order_id": order_id}, goto="verify_order")
    else:
        # No order ID found, ask for it
        response = "To help you with your complaint, I'll need your order ID. Could you please provide it?"
        new_message = {"role": "assistant", "content": response}
        return Command(
            update={"messages": [new_message]},
            goto="__end__"
        )

def verify_order(state: State) -> Command[Literal["ask_details", "__end__"]]:
    """Verify the order ID using the tool."""
    order_id = state["order_id"]
    is_valid = verify_order_id(order_id)
    
    if is_valid:
        response = f"Thank you! I've verified your order ID {order_id}. Please describe your complaint in detail so I can assist you."
        new_message = {"role": "assistant", "content": response}
        return Command(
            update={
                "messages": [new_message],
                "order_id_verified": True
            },
            goto="__end__"
        )
    else:
        response = f"I'm sorry, but the order ID {order_id} appears to be invalid. Please check and provide a valid order ID (should be at least 6 digits)."
        new_message = {"role": "assistant", "content": response}
        return Command(
            update={
                "messages": [new_message],
                "order_id_verified": False,
                "order_id": None  # Reset order ID
            },
            goto="__end__"
        )

def ask_details(state: State) -> Command[Literal["summarize_complaint", "__end__"]]:
    """Check if we have complaint details and summarize or ask for them."""
    # Check if we already have complaint details
    if state.get("complaint_details"):
        return Command(goto="summarize_complaint")
    
    # Check if the user has provided details in the last message
    last_message = None
    for msg in reversed(state["messages"]):
        if isinstance(msg, dict) and msg.get("role") == "user":
            last_message = msg
            break
        elif hasattr(msg, 'type') and msg.type == "human":
            last_message = {"content": msg.content}
            break
    
    if last_message and len(last_message["content"].strip()) > 20:  # Reasonable detail length
        # User has provided details
        complaint_details = last_message["content"]
        return Command(
            update={"complaint_details": complaint_details},
            goto="summarize_complaint"
        )
    else:
        # Ask for details
        response = "Please describe your complaint in detail so I can assist you properly."
        new_message = {"role": "assistant", "content": response}
        return Command(
            update={"messages": [new_message]},
            goto="__end__"
        )

def summarize_complaint(state: State) -> Command[Literal["classify_department", "__end__"]]:
    """Summarize the complaint and ask for confirmation."""
    if state.get("complaint_summary"):
        return Command(goto="classify_department")
    
    # Generate a summary of the complaint
    summary_prompt = f"Summarize the following customer complaint in 1-2 sentences. Be concise but accurate:\n\n{state['complaint_details']}"
    
    response = llm.invoke([HumanMessage(content=summary_prompt)])
    summary = response.content
    
    # Ask for confirmation
    confirmation_message = f"Just to make sure I understand correctly: {summary}\n\nIs this accurate? Please respond with 'yes' or 'no'."
    new_message = {"role": "assistant", "content": confirmation_message}
    
    return Command(
        update={
            "messages": [new_message],
            "complaint_summary": summary
        },
        goto="__end__"
    )

def classify_department(state: State) -> Command[Literal["log_complaint", "__end__"]]:
    """Classify the complaint into a department."""
    # Check if user confirmed the summary
    last_message = None
    for msg in reversed(state["messages"]):
        if isinstance(msg, dict) and msg.get("role") == "user":
            last_message = msg
            break
        elif hasattr(msg, 'type') and msg.type == "human":
            last_message = {"content": msg.content}
            break
    
    if last_message:
        user_response = last_message["content"].lower()
        if any(word in user_response for word in ["yes", "correct", "accurate", "right"]):
            # User confirmed, proceed with classification
            classification_prompt = f"""Classify this complaint into one of these departments:
- Billing: Issues with payments, refunds, charges, or invoices
- Technical Support: Problems with setup, functionality, or technical issues  
- Product Quality: Defective products, wrong items, or quality issues
- General Inquiry: Other non-urgent questions or concerns

Complaint: {state['complaint_details']}"""
            
            classification = classifier_llm.invoke([HumanMessage(content=classification_prompt)])
            department = classification.department
            
            return Command(
                update={"department": department},
                goto="log_complaint"
            )
        else:
            # User didn't confirm, ask for clarification
            response = "I apologize for the misunderstanding. Could you please clarify your complaint?"
            new_message = {"role": "assistant", "content": response}
            return Command(
                update={
                    "messages": [new_message],
                    "complaint_details": None,
                    "complaint_summary": None
                },
                goto="__end__"
            )
    else:
        response = "Please confirm if my summary of your complaint is accurate by responding 'yes' or 'no'."
        new_message = {"role": "assistant", "content": response}
        return Command(
            update={"messages": [new_message]},
            goto="__end__"
        )

def log_complaint(state: State) -> Command[Literal["__end__"]]:
    """Log the complaint to the database."""
    success = log_complaint_to_db(
        order_id=state["order_id"],
        department=state["department"],
        complaint_details=state["complaint_details"]
    )
    
    if success:
        response = f"Thank you for your feedback. I've logged your complaint with our {state['department']} department. We'll get back to you shortly. Is there anything else I can help you with?"
    else:
        response = "I apologize, but there was an error logging your complaint. Please try again later or contact our support team directly."
    
    new_message = {"role": "assistant", "content": response}
    return Command(
        update={
            "messages": [new_message],
            "complaint_logged": success
        },
        goto="__end__"
    )

# Building the graph
def create_complaint_graph():
    """Create and compile the complaint handling graph."""
    graph_builder = StateGraph(State)
    
    # Add nodes
    graph_builder.add_node("check_order_id", check_order_id)
    graph_builder.add_node("verify_order", verify_order)
    graph_builder.add_node("ask_details", ask_details)
    graph_builder.add_node("summarize_complaint", summarize_complaint)
    graph_builder.add_node("classify_department", classify_department)
    graph_builder.add_node("log_complaint", log_complaint)
    
    # Set entry point
    graph_builder.set_entry_point("check_order_id")
    
    # Compile the graph
    return graph_builder.compile()

# Main handler class
class ComplaintHandler:
    def __init__(self):
        self.graph = create_complaint_graph()
        self.state = {
            "messages": [],
            "order_id": None,
            "order_id_verified": None,
            "complaint_details": None,
            "complaint_summary": None,
            "department": None,
            "complaint_logged": None
        }
    
    def handle_message(self, user_message: str) -> str:
        """Handle a user message and return the response."""
        # Add user message to state
        user_msg = {"role": "user", "content": user_message}
        self.state["messages"].append(user_msg)
        
        # Determine which node to start from based on current state
        if not self.state.get("order_id"):
            start_node = "check_order_id"
        elif not self.state.get("order_id_verified"):
            start_node = "verify_order"
        elif not self.state.get("complaint_details"):
            start_node = "ask_details"
        elif not self.state.get("complaint_summary"):
            start_node = "summarize_complaint"
        elif not self.state.get("department"):
            start_node = "classify_department"
        elif not self.state.get("complaint_logged"):
            start_node = "log_complaint"
        else:
            return "Your complaint has been successfully logged. Is there anything else I can help you with?"
        
        # Run the graph from the appropriate node
        try:
            result = self.graph.invoke(self.state)
            self.state.update(result)
            
            # Get the last assistant message
            for msg in reversed(self.state["messages"]):
                if isinstance(msg, dict) and msg.get("role") == "assistant":
                    return msg["content"]
            
            return "I'm ready to help with your complaint. Please provide your order ID."
            
        except Exception as e:
            print(f"Error handling message: {e}")
            return "I apologize, but I encountered an error. Please try again."
    
    def reset(self):
        """Reset the conversation state."""
        self.state = {
            "messages": [],
            "order_id": None,
            "order_id_verified": None,
            "complaint_details": None,
            "complaint_summary": None,
            "department": None,
            "complaint_logged": None
        }






In [6]:
def interactive_mode():
    """Run the system in interactive mode."""
    handler = ComplaintHandler()
    
    print("=== Customer Service Complaint System ===")
    print("Type 'quit' to exit, 'reset' to start over\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'reset':
            handler.reset()
            print("Conversation reset. How can I help you today?\n")
            continue
        elif not user_input:
            continue
        
        try:
            response = handler.handle_message(user_input)
            print(f"Assistant: {response}\n")
        except Exception as e:
            print(f"Error: {e}\n")

In [ ]:
# interactive_mode()

## intent classifier

In [ ]:
# test_conversation()

In [ ]:
# example_conversation()

In [ ]:
# interactive_mode()

## Checking the tools

In [ ]:
import os
import logging as log
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from typing import Optional, Dict, Any

# Load environment variables
load_dotenv()

def get_db_connection():
    """Create and return a database connection."""
    db_user = os.getenv("DB_USER")
    db_password_raw = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    db_name = os.getenv("DB_NAME")

    if not all([db_user, db_password_raw, db_host, db_port, db_name]):
        log.error("Database configuration is missing in .env file.")
        return None

    try:
        encoded_password = quote_plus(db_password_raw)
        connection_url = f"postgresql://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}"
        engine = create_engine(connection_url)
        return engine
    except Exception as e:
        log.error(f"Failed to create database connection: {e}")
        return None

def verify_order_id(order_id: str) -> Dict[str, Any]:
    """
    Verify if an order ID exists in the purchase_history table.
    Returns a dictionary with verification status and order details.
    """
    log.info(f"Verifying order ID: {order_id}")
    
    engine = get_db_connection()
    if not engine:
        return {"valid": False, "error": "Database connection failed"}
    
    try:
        with engine.connect() as connection:
            # Query to check if order exists and get user details
            query = text("""
                SELECT ph.order_id, ph.asin, ph.purchase_date, u.name, u.email, u.id as user_id
                FROM purchase_history ph
                JOIN users u ON ph.user_id = u.id
                WHERE ph.order_id = :order_id
            """)
            
            result = connection.execute(query, {"order_id": order_id})
            order_data = result.mappings().first()
            
            if order_data:
                log.info(f"Order ID {order_id} verified successfully")
                return {
                    "valid": True,
                    "order_id": order_data["order_id"],
                    "asin": order_data["asin"],
                    "purchase_date": order_data["purchase_date"].isoformat() if order_data["purchase_date"] else None,
                    "customer_name": order_data["name"],
                    "customer_email": order_data["email"],
                    "user_id": order_data["user_id"]
                }
            else:
                log.warning(f"Order ID {order_id} not found in database")
                return {"valid": False, "error": "Order ID not found"}
                
    except Exception as e:
        log.error(f"Error verifying order ID {order_id}: {e}")
        return {"valid": False, "error": f"Database error: {str(e)}"}
    
    finally:
        engine.dispose()

def log_complaint_to_db(order_id: str, department: str, complaint_details: str, complaint_summary: str) -> Dict[str, Any]:
    """
    Log a complaint to the support_tasks table in the database.
    Returns a dictionary with logging status and task details.
    """
    log.info(f"Logging complaint for order {order_id} to {department} department")
    
    # First verify the order exists
    order_verification = verify_order_id(order_id)
    if not order_verification["valid"]:
        return {
            "success": False,
            "error": f"Cannot log complaint: {order_verification.get('error', 'Order verification failed')}",
            "task_id": None
        }
    
    engine = get_db_connection()
    if not engine:
        return {"success": False, "error": "Database connection failed", "task_id": None}
    
    try:
        with engine.connect() as connection:
            # Get user_id from order verification
            user_id = order_verification["user_id"]
            
            # Find an appropriate team member for the department
            team_member_query = text("""
                SELECT tm.id, tm.name, tm.email
                FROM team_members tm
                JOIN departments d ON tm.department_id = d.id
                WHERE d.name ILIKE :department
                LIMIT 1
            """)
            
            team_member_result = connection.execute(
                team_member_query, 
                {"department": f"%{department}%"}
            )
            team_member = team_member_result.mappings().first()
            
            assigned_to = team_member["id"] if team_member else None
            
            # Insert the complaint into support_tasks
            insert_query = text("""
                INSERT INTO support_tasks 
                (user_id, order_id, assigned_to_member_id, summary, department, status)
                VALUES (:user_id, :order_id, :assigned_to, :summary, :department, 'open')
                RETURNING id, created_at
            """)
            
            result = connection.execute(
                insert_query,
                {
                    "user_id": user_id,
                    "order_id": order_id,
                    "assigned_to": assigned_to,
                    "summary": complaint_summary,
                    "department": department
                }
            )
            
            task_data = result.mappings().first()
            connection.commit()
            
            log.info(f"Complaint logged successfully for order {order_id}, task ID: {task_data['id']}")
            
            return {
                "success": True,
                "task_id": task_data["id"],
                "created_at": task_data["created_at"].isoformat(),
                "assigned_to": team_member["name"] if team_member else "Unassigned",
                "department": department,
                "order_details": order_verification
            }
            
    except Exception as e:
        log.error(f"Error logging complaint for order {order_id}: {e}")
        connection.rollback()
        return {"success": False, "error": f"Database error: {str(e)}", "task_id": None}
    
    finally:
        engine.dispose()



## joining these 2

In [7]:
from typing import Literal, TypedDict, Dict, Any, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.types import Command
from pydantic import BaseModel

# Import your existing modules (adjust imports based on your file structure)
# from your_complaint_module import ComplaintHandler
from agent import run_agent

class UserIntent(BaseModel):
    """The user's current intent in the conversation"""
    intent: Literal["complaint", "qna"]

class RouterState(TypedDict):
    """Router state for managing conversation flow."""
    messages: List[Dict[str, str]]
    current_intent: str | None
    complaint_handler_state: Dict[str, Any] | None
    conversation_active: bool

class IntentRouter:
    """Main router class that manages intent classification and routing."""
    
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.router_llm = self.llm.with_structured_output(UserIntent)
        self.complaint_handler = None  # Will be initialized when needed
        self.state = {
            "messages": [],
            "current_intent": None,
            "complaint_handler_state": None,
            "conversation_active": True
        }
        
        # Route instructions for the classifier
        self.route_instructions = """You are managing a customer service system that handles two types of requests:

(1) Complaints: When customers have issues with orders, billing problems, technical issues, 
    product quality concerns, or want to file formal complaints that need to be logged and tracked.

(2) Q&A: General questions about products (especially beauty products), product recommendations, 
    comparisons, general conversation, greetings, or informational queries.

Based on the conversation context, determine the user's intent:

- Return 'complaint' if they are reporting a problem, have an issue with an order, 
  want to file a complaint, or need assistance with a specific problem that requires logging.
  
- Return 'qna' if they are asking general questions, seeking product information, 
  making casual conversation, or need general assistance.

Important: Once a user is in 'complaint' mode and actively working through the complaint process,
they should remain in complaint mode until the complaint is fully resolved or they explicitly 
change the topic to something unrelated to their complaint.

Important: After a complaint is resolved, the user can switch back to 'qna' mode for general questions or a New 'complaint' mode depending on context.

Do NOT try to respond to the user. Just classify the intent."""

    def classify_intent(self, messages: List[Dict[str, str]]) -> str:
        """Classify user intent based on conversation history."""
        # If we're already in complaint mode and the complaint isn't resolved
        if (self.state.get("current_intent") == "complaint" and 
            self.complaint_handler and 
            not self.complaint_handler.state.get("complaint_logged")):
            
            # Check if user is trying to change topic completely
            last_message = messages[-1]["content"].lower() if messages else ""
            topic_change_indicators = [
                "nevermind", "forget it", "cancel", "different question", 
                "something else", "new question", "change topic"
            ]
            
            if any(indicator in last_message for indicator in topic_change_indicators):
                return "qna"  # Allow topic change
            else:
                return "complaint"  # Continue with complaint
        
        # Use LLM to classify intent
        system_msg = {"role": "system", "content": self.route_instructions}
        all_messages = [system_msg] + messages
        
        try:
            response = self.router_llm.invoke(all_messages)
            return response.intent
        except Exception as e:
            print(f"Error in intent classification: {e}")
            return "qna"  # Default to Q&A on error

    def handle_complaint(self, user_message: str) -> str:
        """Handle complaint using the existing ComplaintHandler."""
        if not self.complaint_handler:
            # Import and initialize complaint handler when needed
            try:
                self.complaint_handler = ComplaintHandler()
            except ImportError:
                return "I apologize, but the complaint system is currently unavailable. Please try again later."
        
        return self.complaint_handler.handle_message(user_message)

    def handle_qna(self, user_message: str) -> str:
        """Handle Q&A using the existing agent."""
        try:
            from agent import run_agent  # Adjust import path
            return run_agent(user_message)
        except ImportError:
            return "I apologize, but I'm having trouble accessing the Q&A system. Please try again later."

    def process_message(self, user_message: str) -> str:
        """Main method to process user messages and return responses."""
        # Add user message to conversation history
        user_msg = {"role": "user", "content": user_message}
        self.state["messages"].append(user_msg)
        
        # Classify intent
        classified_intent = self.classify_intent(self.state["messages"])
        
        # Handle based on intent
        if classified_intent == "complaint":
            self.state["current_intent"] = "complaint"
            response = self.handle_complaint(user_message)
            
            # Check if complaint is completed
            if (self.complaint_handler and 
                self.complaint_handler.state.get("complaint_logged")):
                # Complaint is resolved, can switch to other intents
                pass
                
        else:  # qna
            # If switching from complaint to qna, reset complaint handler
            if self.state.get("current_intent") == "complaint":
                if self.complaint_handler:
                    self.complaint_handler.reset()
                    self.complaint_handler = None
            
            self.state["current_intent"] = "qna"
            response = self.handle_qna(user_message)
        
        # Add assistant response to conversation history
        assistant_msg = {"role": "assistant", "content": response}
        self.state["messages"].append(assistant_msg)
        
        return response

    def reset_conversation(self):
        """Reset the entire conversation state."""
        if self.complaint_handler:
            self.complaint_handler.reset()
        self.complaint_handler = None
        self.state = {
            "messages": [],
            "current_intent": None,
            "complaint_handler_state": None,
            "conversation_active": True
        }

    def get_conversation_state(self) -> Dict[str, Any]:
        """Get current conversation state for debugging/monitoring."""
        return {
            "current_intent": self.state.get("current_intent"),
            "message_count": len(self.state.get("messages", [])),
            "complaint_active": self.complaint_handler is not None,
            "complaint_logged": (self.complaint_handler.state.get("complaint_logged") 
                               if self.complaint_handler else None)
        }



def create_intent_router_graph():
    """Create a LangGraph implementation of the intent router."""
    
    def intent_classifier(state: RouterState) -> Command[Literal["complaint_handler", "qna_handler"]]:
        """Classify user intent and route to appropriate handler."""
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        router_llm = llm.with_structured_output(UserIntent)
        
        route_instructions = """You are managing a customer service system that handles two types of requests:
        (1) Complaints: Issues with orders, billing, technical problems, formal complaints
        (2) Q&A: General questions, product info, recommendations, casual conversation
        
        Return 'complaint' for problem reports or 'qna' for general questions."""
        
        system_msg = {"role": "system", "content": route_instructions}
        messages = [system_msg] + [{"role": m["role"], "content": m["content"]} 
                                  for m in state["messages"]]
        
        response = router_llm.invoke(messages)
        
        return Command(
            update={"current_intent": response.intent},
            goto=f"{response.intent}_handler"
        )
    
    def complaint_handler_node(state: RouterState) -> Command[Literal["__end__"]]:
        """Handle complaints using ComplaintHandler."""
        # This would use your ComplaintHandler
        # Implementation similar to the class-based approach above
        pass
    
    def qna_handler_node(state: RouterState) -> Command[Literal["__end__"]]:
        """Handle Q&A using the agent."""
        # This would use your run_agent function
        # Implementation similar to the class-based approach above
        pass
    
    # Build the graph
    graph_builder = StateGraph(RouterState)
    graph_builder.add_node("intent_classifier", intent_classifier)
    graph_builder.add_node("complaint_handler", complaint_handler_node)
    graph_builder.add_node("qna_handler", qna_handler_node)
    
    graph_builder.set_entry_point("intent_classifier")
    graph_builder.add_edge("complaint_handler", END)
    graph_builder.add_edge("qna_handler", END)
    
    return graph_builder.compile()


# Usage example and testing

router = IntentRouter()

print("Customer Service Router - Type 'quit' to exit")
print("-" * 50)

while True:
    user_input = input("\nUser: ").strip()
    
    if user_input.lower() in ['quit', 'exit', 'bye']:
        break
        
    if user_input.lower() == 'reset':
        router.reset_conversation()
        print("Conversation reset.")
        continue
        
    if user_input.lower() == 'status':
        print("Current state:", router.get_conversation_state())
        continue
    
    if user_input:
        try:
            response = router.process_message(user_input)
            print(f"Assistant: {response}")
            print(f"[Intent: {router.state.get('current_intent')}]")
        except Exception as e:
            print(f"Error: {e}")


# Test cases for validation
def test_intent_router():
    """Test the intent router with various scenarios."""
    router = IntentRouter()
    
    test_cases = [
        ("Hello, how are you?", "qna"),
        ("I have a problem with my order 123456", "complaint"),
        ("What products do you recommend?", "qna"),
        ("My order is damaged and I want a refund", "complaint"),
        ("Can you tell me about skincare products?", "qna"),
        ("I want to file a complaint about billing", "complaint"),
    ]
    
    print("Testing Intent Classification:")
    print("-" * 40)
    
    for message, expected in test_cases:
        router.state["messages"] = [{"role": "user", "content": message}]
        classified = router.classify_intent(router.state["messages"])
        status = "✓" if classified == expected else "✗"
        print(f"{status} '{message}' -> {classified} (expected: {expected})")
        router.reset_conversation() 



Customer Service Router - Type 'quit' to exit
--------------------------------------------------
[ 2025-08-22 17:10:44,182 ] httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[ 2025-08-22 17:10:44,918 ] httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Assistant: Good morning! How can I help you today?
[Intent: qna]
[ 2025-08-22 17:11:05,613 ] httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Assistant: To help you with your complaint, I'll need your order ID. Could you please provide it?
[Intent: complaint]
[ 2025-08-22 17:11:26,675 ] root - INFO - Verifying order ID: 13214155
[ 2025-08-22 17:11:26,723 ] root - WARNING - Order ID 13214155 not found in database
Assistant: I'm sorry, but the order ID 13214155 appears to be invalid. Please check and provide a valid order ID (should be at least 6 digits).
[Intent: complaint]
Assistant: To help you with you

In [ ]:
test_intent_router()

In [ ]:
# interactive_mode()

In [ ]:
# interactive_run()